In [9]:
import mediapipe as mp
import cv2

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [11]:
def distance(p1, p2):
    return ((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2 + (p2.z - p1.z) ** 2) ** 0.5

In [12]:
def finger_raised(landmarks):
    fingers = {
        'Pulgar': False,
        'Indice': False,
        'Medio': False,
        'Anular': False,
        'Menique': False
    }

    # Pulgar: comparar eje X (diferente si es mano izquierda o derecha)
    # Si la palma está mirando al frente, comparamos 4 y 3
    if landmarks[17].x < landmarks[5].x:  
        fingers['Pulgar'] = landmarks[4].x > landmarks[3].x
    else:  
        fingers['Pulgar'] = landmarks[4].x < landmarks[3].x
    
    fingers['Indice'] = landmarks[8].y < landmarks[6].y
    fingers['Medio'] = landmarks[12].y < landmarks[10].y
    fingers['Anular'] = landmarks[16].y < landmarks[14].y
    fingers['Menique'] = landmarks[20].y < landmarks[18].y

    return fingers

In [13]:
def fingers_together():
    return distance(landmarks[8], landmarks[12]) < 0.13 and distance(landmarks[12], landmarks[16]) < 0.13 and distance(landmarks[16], landmarks[20]) < 0.13    

In [14]:
# Gesture detection function
def gesture_detection(fingers):
    # Predefined gestures
    if all(fingers.values()) and fingers_together():
        return 'Alto ✋'
    elif fingers['Menique'] and fingers['Indice'] and not any([fingers['Pulgar'], fingers['Medio'], fingers['Anular']]):
        return 'Cuernos 🤘'
    elif fingers['Pulgar'] and fingers['Menique'] and not any([fingers['Anular'], fingers['Medio'], fingers['Indice']]):
        return 'Llámame 🤙'
    elif fingers['Pulgar'] and fingers['Menique'] and fingers['Indice'] and not any ([fingers['Medio'], fingers['Anular']]):
        return '"Te quiero" 🤟'
    # abs -> Valor absoluto
    elif fingers['Indice'] and fingers['Medio'] and fingers['Anular'] and fingers['Menique'] and abs(landmarks[12].x - landmarks[16].x) > 0.06 and abs(landmarks[8].x - landmarks[12].x) < 0.05 and abs(landmarks[16].x - landmarks[20].x) < 0.05:
        return 'Saludo Vulcano 🖖'
    elif not all(fingers.values()):
        return 'Puño 👊'
    elif fingers['Pulgar'] and fingers['Indice'] and not any([fingers['Medio'], fingers['Anular'], fingers['Menique']]) and distance(landmarks[4], landmarks[8]) < 0.09:
        return 'Pinza 🤏'   
    elif fingers['Indice'] and not any([fingers['Pulgar'], fingers['Anular'], fingers['Medio'], fingers['Menique']]):
        if landmarks[8].x < landmarks[5].x:
            return 'Señalar a la izquierda 👈'
        else:
            return 'Señalar a la derecha 👉'
    elif fingers['Indice'] and fingers['Pulgar'] and not any([fingers['Anular'], fingers['Medio'], fingers['Menique']]):
        if distance(landmarks[4], landmarks[8]) < 0.50:
            return 'Corazón Coreano'
    elif fingers['Indice'] and fingers['Medio'] and not any([fingers['Pulgar'], fingers['Anular'], fingers['Menique']]):
        return 'Victory ✌️'
    elif fingers['Pulgar'] and not any([fingers['Indice'], fingers['Medio'], fingers['Anular'], fingers['Menique']]):
        return 'Pulgar arriba 👍'
    elif any([fingers['Medio'], fingers['Anular'], fingers['Menique']]) and distance(landmarks[4], landmarks[8]) < 0.05:
        return 'Ok 👌'
    elif all(fingers.values()):
        return 'Mano abierto 🖐️'
    else:
        return 'Gesto no definido'

In [15]:
cap = cv2.VideoCapture(0)

In [8]:
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence = 0.7) as hands:
   
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Flip the image horizontally for a later selfie-view display
    image = cv2.flip(image, 1)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        # Draw hand landmarks
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        landmarks = hand_landmarks.landmark

        fingers = finger_raised(landmarks)
        gesture = gesture_detection(fingers)

        # Show gesture detection
        cv2.putText(image, f'Gesture: {gesture}', (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Option
        info_fingers = f'Fingers: {[k for k, v in fingers.items() if v]}'
        cv2.putText(image, info_fingers, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (25, 255, 0), 2)

        if gesture == 'Corazón Coreano':
            cap.release()
            cv2.destroyAllWindows()
            exit()

    cv2.imshow('Gesture Tracking', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()